In [22]:
import json
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [28]:
def write_separate_genotype_fastas(serotype):
    #find which genotype each strain belongs to
    genotype_file = f'../results/clades_{serotype}.json'

    genotype_by_node = {}

    with open(genotype_file, "r") as genotype_handle:
        genotypes = json.load(genotype_handle)
        for node, v in genotypes['nodes'].items():
            if 'NODE' not in node:
                genotype_by_node[node] = v['clade_membership']
                
    #write a fasta data file for each genotype
    data_file = f'../data/dengue_{serotype}.fasta'

    seq_records_by_genotype = {}

    with open(data_file, 'r') as data_handle:
        for virus in SeqIO.parse(data_handle, 'fasta'):
            virus_id = virus.id.split('|')[0]
            if virus_id in genotype_by_node.keys():
                genotype_of_node = genotype_by_node[virus_id]
                if genotype_of_node in seq_records_by_genotype.keys():
                    seq_records_by_genotype[genotype_of_node] += [SeqRecord(virus.seq, id=virus.id, 
                                                                            description=virus.description)]
                else:
                    seq_records_by_genotype[genotype_of_node] = [SeqRecord(virus.seq, id=virus.id, 
                                                                            description=virus.description)]
    for g, recs in seq_records_by_genotype.items():
        if g!= 'unassigned':
            genotype = g.split('/')[1]
            genotype_data_filename = f'data/dengue_{serotype}_{genotype}.fasta'

            SeqIO.write(recs, genotype_data_filename, "fasta")
    

In [30]:
write_separate_genotype_fastas('denv1')

In [29]:
write_separate_genotype_fastas('denv2')

In [31]:
write_separate_genotype_fastas('denv3')

In [32]:
write_separate_genotype_fastas('denv4')